In [21]:
from ipywidgets import Layout
import ipywidgets as widgets
import ipyvuetify as v
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import FuncFormatter, MultipleLocator
import requests
from IPython.display import display, clear_output, Markdown
import json

<div style="
    display: flex;
    justify-content: space-between;
    align-items: center;
    height: 30vh;
    background-color: #ffbf69;
    border: 3px solid #140c00;
    border-radius: 3%;
">
    <!-- List of names centered to the left -->
    <div style="margin-right: 20px; display: flex; align-items: center;">
        <ul style="list-style-type: none; text-align: left; font: 1.2em/2; font-family: 'Fira Sans', sans-serif;">
            <li>Eithan Capaella</li>
            <li>Pedro Pagan</li>
            <li>Sebastian Estrada</li>
            <li>Sebastian Caballero</li>
            <li>Yariel Mercado</li>
        </ul>
    </div>
    <!-- Image in the center -->
    <div>
        <img src="sqlYTES.png" style="width: 300px; height: 200px; margin: 0 20px;">
    </div>
    <!-- Text spans centered to the right -->
    <div style="text-align: right; display: flex; flex-direction: column; align-items: center; margin-right: 30px;">
        <span style="font: 1.2em/2; font-family: 'Fira Sans', sans-serif;">CIIC 4060</span><br>
        <span style="font: 1.2em/2; font-family: 'Fira Sans', sans-serif;">Data Base Systems</span>
    </div>
</div>

<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    height: 10vh;">
    <h1>Choose which requirement you want to view: </h1>
</div>

In [ ]:
backend = 'http://127.0.0.1:5000/sqlytes/'

In [ ]:
class WarehouseWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses().get('Warehouses')
        self.current_warehouse_index = 0
        self.all_warehouses_df = pd.DataFrame()
       
        self.warehouses_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All parts for a warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )


        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        
    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids
    
    def getAllWarehouses(self):
        """Return all warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextWarehouse(self,_):
        self.current_warehouse_index = (self.current_warehouse_index+1) % len(self.warehouses)
        print(f"Displaying index {self.current_warehouse_index}")
        selected_warehouse = self.warehouses[self.current_warehouse_index]
        print(f"Displaying Warehouse ID: {selected_warehouse}")
        selected_warehouse_df = pd.DataFrame.from_dict(selected_warehouse, orient="index")
        self.all_warehouses_df = selected_warehouse_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_warehouses_df)
            
    def getWarehousePartsAPI(self, selected_warehouse):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'warehouse/{selected_warehouse}/parts'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # clear cell
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["WarehouseParts"])
                display(all_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse has no parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

    def on_selection_change(self, change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getWarehousePartsAPI(selected_warehouse)


In [ ]:
def displayWarehouseWidgets():
    warehouseWidgets = WarehouseWidgets()
    warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
    warehouseWidgets.warehouse_selection.observe(warehouseWidgets.on_selection_change)
    display(warehouseWidgets.warehouse_selection)
    display(warehouseWidgets.output_area)

In [ ]:
class SuppliedPartsWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_suppliers = self.getAvailableSuppliers()
        self.suppliers = self.getAllSuppliers()
        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
        self.supplier_selection = widgets.Dropdown(
            options=self.available_suppliers,
            value=self.available_suppliers[0],
            description="Select sid:"
        )

        self.supplier_selection.observe(self.on_selection_change)


    def getAvailableSuppliers(self):
        """Return all IDs of the suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # List to store all available sid's.
        sids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data:
                sids.append(record['id'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sids

    def getAllSuppliers(self):
        """Return all suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}


    def getSuppliedPartsAPI(self,selected_supplier):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'supplies/{selected_supplier}'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data)
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSuppliedPartsAPI(selected_supplier)

In [ ]:
def displaySuppliedParts():
    suppliedPartsWidgets = SuppliedPartsWidgets()
    suppliedPartsWidgets.getSuppliedPartsAPI(suppliedPartsWidgets.supplier_selection.value)
    suppliedPartsWidgets.supplier_selection.observe(suppliedPartsWidgets.on_selection_change)
    display(suppliedPartsWidgets.supplier_selection)
    display(suppliedPartsWidgets.output_area)

In [ ]:
class TransactionWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses()
        self.current_warehouse_index = 0
        self.all_transactions_df = pd.DataFrame()
       
        self.warehouse_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All transactions in this warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        

    def getAvailableWarehouses(self):
        """
        Return all IDs of the warehouses in the DB
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available sid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sorted(wids)

    
    def getAllWarehouses(self):
        """
        Return all suppliers in the Data Base.
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
                
    def getTransactionsPerWarehouseAPI(self, selected_warehouse):
        endpoint = f'transaction'
        response = requests.put(f'{backend}{endpoint}', data=json.dumps({"wid": selected_warehouse}), headers={"Content-Type": "application/json"})
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data["Result"])
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse does not have any transactions'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse    = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getTransactionsPerWarehouseAPI(selected_warehouse)


In [ ]:
def displayTransactionWidget():
    transactionWidget = TransactionWidgets()
    transactionWidget.getTransactionsPerWarehouseAPI(transactionWidget.warehouse_selection.value)
    transactionWidget.warehouse_selection.observe(transactionWidget.on_selection_change)
    display(transactionWidget.warehouse_selection)
    display(transactionWidget.output_area)

In [ ]:
class PartPricesWidget:
    def __init__(self):
        self.table_output_area = widgets.Output()
        self.graph_output_area = widgets.Output()
        self.df = pd.DataFrame([])


    def getAllParts(self):
        endpoint = f'part'
        response = requests.get(f'{backend}{endpoint}')
        with self.table_output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                self.df = pd.DataFrame(data)
                display(self.df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

    def graphAllParts(self):
        names = []
        prices = []
        for _, row in self.df.iterrows():
            names.append(row['Name'])
            prices.append(row['msrp'])
        with self.graph_output_area:
            plt.figure(figsize=(16, 6))
            plt.bar(names, prices, color='red')
            plt.xlabel('Part Names')
            plt.ylabel('Part Prices ($)')
            plt.title('Price of all Parts')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()

In [ ]:
def displayPartPriceWidget():
    partPricesWidget = PartPricesWidget()
    partPricesWidget.getAllParts()
    partPricesWidget.graphAllParts()
    display(partPricesWidget.table_output_area)
    display(partPricesWidget.graph_output_area)

In [ ]:
class MostRacksWidgets:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_most_racks(self):
        """Obtain Top 10 Warehouses with the most Racks.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/rack'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Top Racks per Warehouse"])
                display(all_parts_df)

                                #Data for Bar Graph
                incoming = [entry["Rack Count"]  for entry in data["Top Racks per Warehouse"]]
                warehouse = [entry["Warehouse"] for entry in data["Top Racks per Warehouse"]]

                #Creating Bar Graph
                plt.bar(warehouse, incoming, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Rack Count')
                plt.title('Racks per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()


            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class MostIncomingTransactions:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_most_incoming(self):
        """Obtain Top 5 Warehouses with the most incoming transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/incoming'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Incoming Transactions"])
                display(all_parts_df)

                #Data for Bar Graph
                incoming = [entry["Total Incoming Transactions"]  for entry in data["Most Incoming Transactions"]]
                warehouse = [entry["Warehouse"] for entry in data["Most Incoming Transactions"]]

                #Creating Bar Graph
                plt.bar(warehouse, incoming, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Incoming')
                plt.title('Incoming Transactions per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()

            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class MostExchangesWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_most_delivered(self):
        """Obtain Top 5 Warehouses that deliver the most exchanges.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/deliver'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Transfers"])
                display(all_parts_df)

                #Data for Bar Graph
                transfers = [entry["Total Transfers"]  for entry in data["Most Transfers"]]
                warehouse = [entry["Warehouse"] for entry in data["Most Transfers"]]

                #Creating Bar Graph
                plt.bar(warehouse, transfers, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Exchanges')
                plt.title('Exchanges per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()

            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class MostUserTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_most_users(self):
        """Obtain Top 3 Users that made the most transactions..

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/transactions'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Top User Transactions"])
                display(all_parts_df)

                #Data for Bar Graph
                transactions = [entry["Transaction Count"]  for entry in data["Top User Transactions"]]
                name = [entry["First Name"] + " " + entry["Last Name"] for entry in data["Top User Transactions"]]

                #Creating Bar Graph
                plt.bar(name, transactions, color='#95d0ed')
                plt.xlabel('\n Users')
                plt.ylabel('Total Transactions')
                plt.title('Transactions per User')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()

            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class LeastOutTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_least_outgoing(self):
        """Obtain Top 3 warehouses with the least outgoing transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'least/outgoing'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Least Outgoing Transactions"])
                display(all_parts_df)

                #Data for Bar Graph
                outgoing = [entry["Total Outgoing Transactions"]  for entry in data["Least Outgoing Transactions"]]
                warehouse = [entry["Warehouse"] for entry in data["Least Outgoing Transactions"]]

                #Creating Bar Graph
                plt.bar(warehouse, outgoing, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Outgoing Transactions')
                plt.title('Least Outgoing Transaction per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()

            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class MostCityTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()


    def get_most_cities(self):
        """Obtain Top 3 warehouses' cities with the most transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/city'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200:
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Transactions per City"])
                display(all_parts_df)

                #Data for Bar Graph
                transaction_Counts = [entry["Total Transactions"]  for entry in data["Most Transactions per City"]]
                cities = [entry["Warehouse City"] for entry in data["Most Transactions per City"]]

                #Creating Bar Graph
                plt.bar(cities, transaction_Counts, color='#95d0ed')
                plt.xlabel('\n Cities')
                plt.ylabel('Total Transactions')
                plt.title('Total Transactions per City')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()


            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
def displayGlobalStatisticsWidget():
    display(Markdown("## Top 10 Warehouses with the most Racks"))
    mostRacksWidgets = MostRacksWidgets()
    mostRacksWidgets.get_most_racks()
    display(mostRacksWidgets.output_area)

    display(Markdown("## Top 5 Warehouses with the most Incoming Transactions"))
    mostIncomingTransactions = MostIncomingTransactions()
    mostIncomingTransactions.get_most_incoming()
    display(mostIncomingTransactions.output_area)

    display(Markdown("## Top 5 Warehouses that deliver the most Exchanges"))
    mostExchangesWidget = MostExchangesWidget()
    mostExchangesWidget.get_most_delivered()
    display(mostExchangesWidget.output_area)

    display(Markdown("## Top 3 Users that made the most Transactions"))
    mostUserTransactionsWidget = MostUserTransactionsWidget()
    mostUserTransactionsWidget.get_most_users()
    display(mostUserTransactionsWidget.output_area)

    display(Markdown("## Top 3 Warehouses with the least Outgoing Transactions"))
    leastOutTransactionsWidget = LeastOutTransactionsWidget()
    leastOutTransactionsWidget.get_least_outgoing()
    display(leastOutTransactionsWidget.output_area)

    display(Markdown("## Top 3 Warehouses' Cities with the most Transactions"))
    mostCityTransactionsWidget = MostCityTransactionsWidget()
    mostCityTransactionsWidget.get_most_cities()
    display(mostCityTransactionsWidget.output_area)

In [ ]:
# For graphs
def currency_formatter(float, pos):
    return f"${float:.2f}"

In [ ]:
class LocalStatsWidgets:
    def __init__(self):
        self.yearly_output = widgets.Output()
        self.capacity_output = widgets.Output()
        self.material_output = widgets.Output()
        self.expensive_output = widgets.Output()
        self.suppliers_output = widgets.Output()
        self.days_output = widgets.Output()
        self.receives_output = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.outputs = {self.yearly_output: self.getYearlyProfit,
                        self.capacity_output: self.getCapacityUnderThresh,
                        self.material_output: self.getBottomMaterials,
                        self.expensive_output: self.getExpensiveRacks,
                        self.suppliers_output: self.getTopSuppliers,
                        self.days_output: self.getSmallestIncomingCost,
                        self.receives_output: self.getMostExchanges}

        self.wid_input = widgets.Text(value="", placeholder="Enter warehouse ID", description="Enter wid:", disabled=False)
        self.uid_input = widgets.Text(value="", placeholder="Enter User ID", description="Enter uid:", disabled=False)
        self.submit_button = widgets.Button(description="Run query")

    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids

    def validate_input(self):
        try:
            wid, uid = int(self.wid_input.value), int(self.uid_input.value)
        except:
            return -1, -1
        return wid, uid

    def user_works(self, wid, uid):
        response = requests.get(f'{backend}{"warehouse/"}{uid}/worksin/{wid}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        display(f'Error {response.status_code}: Could not get data from the backend.')
        return {}

    def validateLocalStatsConnection(self, response, wid, uid, url):
        if response.status_code == 404:
            if wid not in self.available_warehouses:
                display(f"Warehouse {wid} not valid.")
            else:
                can_access = self.user_works(wid, uid)
                if not can_access:
                    display(f'Error {response.status_code}: User {uid} cannot work in Warehouse #{wid}.')
                else:
                    display(f'Error {response.status_code}: Warehouse #{wid} has no relevant data.')
        else:
            display(f'{url} accessed with user #{uid}.')
            display(f'Error {response.status_code}: Could not get data from the backend.')

    def getYearlyProfit(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/profit'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.yearly_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                sorted_data = sorted(data['Yearly Profit'], key=lambda x: x['Year'])
                years = [entry['Year'] for entry in sorted_data]
                net_profits = [entry['Net Profit'] for entry in sorted_data]
                plt.plot(years, net_profits, marker='o', linestyle='-')
                plt.xlabel('Year')
                plt.ylabel('Net Profit')
                plt.title(f'Yearly Profit for Warehouse #{selected_warehouse}')
                # Formats money
                plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getCapacityUnderThresh(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/rack/lowstock'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.capacity_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                racks = [entry['Rack'] for entry in data['Lowest Threshold Racks']]
                thresholds = [float(entry['Low Capacity Threshold']) for entry in data['Lowest Threshold Racks']]
                parts_quantity = [entry['Parts Quantity'] for entry in data['Lowest Threshold Racks']]
                fig, ax = plt.subplots()
                bars = ax.bar(racks, parts_quantity)
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Threshold')
                ax.set_xticks(range(len(racks)))
                ax.set_xticklabels(racks, rotation=45, ha='right')
                for bar, part_quantity in zip(bars, parts_quantity):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2, height, str(part_quantity), ha='center', va='bottom')
                max_threshold = max(thresholds)
                ax.set_ylim(0, max_threshold)
                num_major_ticks = 5
                ax.yaxis.set_major_locator(MultipleLocator(max_threshold/(num_major_ticks-1)))
                plt.title('Top 5 Racks Under the 25% Capacity Threshold')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getBottomMaterials(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/rack/material'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.material_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                parts_data = data['Bottom Racks']
                rack_names = [f"{part['Part']}" for part in parts_data]
                part_counts = [part['Part Count'] for part in parts_data]
                types = list(set(part['Type'] for part in parts_data))
                # Creating a color map for each rack type
                colors = [cm.tab10(i) for i in range(len(types))]
                fig, ax = plt.subplots()
                for i, type_name in enumerate(types):
                    indices = [j for j, part in enumerate(parts_data) if part['Type'] == type_name]
                    ax.bar([rack_names[j] for j in indices], [part_counts[j] for j in indices], color=colors[i], label=type_name)
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Part Count')
                plt.title('Bottom 3 Part Counts per Type, in Racks')
                plt.xticks(rotation=45, ha='right')
                plt.legend(title='Type', bbox_to_anchor=(1, 1), loc='upper left')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getExpensiveRacks(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/rack/expensive'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.expensive_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                racks_data = data['Most Expensive Racks']
                rack_names = [rack['Rack'] for rack in racks_data]
                rack_prices = [rack['Rack Price'] for rack in racks_data]
                fig, ax = plt.subplots()
                bars = ax.bar(rack_names, rack_prices)
                for bar, rack_price in zip(bars, rack_prices):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2, height, f'${rack_price:.2f}', ha='center', va='bottom')
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Rack Prices')
                plt.title('Top 5 Most Expensive Racks')
                plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getTopSuppliers(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/transaction/suppliers'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.suppliers_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                suppliers_data = data['Top Warehouse Suppliers']
                supplier_names = [supplier['Supplier Name'] for supplier in suppliers_data]
                supply_counts = [supplier['Supply Count'] for supplier in suppliers_data]
                fig, ax = plt.subplots()
                ax.bar(supplier_names, supply_counts)
                ax.set_xlabel('Supplier Names')
                ax.set_ylabel('Supply Count')
                plt.title(f'Top 3 Suppliers for Warehouse #{selected_warehouse}')
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getSmallestIncomingCost(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/transaction/leastcost'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.days_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                costs_data = data['Least Incoming Trans. Costs']
                dates = [datetime.strptime(entry['Transaction Date'], '%a, %d %b %Y %H:%M:%S %Z') for entry in costs_data]
                total_costs = [entry['Total Incoming Cost'] for entry in costs_data]
                fig, ax = plt.subplots()
                ax.plot(dates, total_costs, marker='o', linestyle='-')
                ax.set_xlabel('Transaction Dates')
                ax.set_ylabel('Total Incoming Costs')
                ax.tick_params(axis='x', rotation=30)
                # Setting x-axis, do something when there isn't enough data
                date_range = max(dates) - min(dates)
                if date_range.days > 1:
                    ax.set_xlim(min(dates), max(dates))
                else:
                    ax.set_xlim(min(dates) - pd.DateOffset(days=1), max(dates) + pd.DateOffset(days=1))
                plt.title('Top 3 Days with Smallest Incoming Transaction Costs over Time')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getMostExchanges(self):
        selected_warehouse, selected_user = self.validate_input()
        endpoint = f'warehouse/{selected_warehouse}/users/receivesmost'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.receives_output:
            clear_output(wait=True)
            if selected_warehouse == -1 or selected_user == -1: display("Error, invalid values received when processing the text fields.")
            if response.status_code == 200:
                data = response.json()
                # Plot
                user_data = data['Most User Exchanges']
                user_names = [f"{user['First Name']} {user['Last Name']}" for user in user_data]
                transfer_counts = [user['Transfer Count'] for user in user_data]
                fig, ax = plt.subplots()
                ax.bar(user_names, transfer_counts)
                ax.set_xlabel('Users')
                ax.set_ylabel('Transfer Count')
                plt.title('Top 3 Users that Receive the Most Exchanges')
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

In [ ]:
localStatsWidget = LocalStatsWidgets()

In [ ]:
def display_all_stats(b):
    for output in localStatsWidget.outputs:
        execute = localStatsWidget.outputs.get(output)
        with output:
            execute()

In [ ]:
def displayLocalStatsWidget():
    display(localStatsWidget.wid_input)
    display(localStatsWidget.uid_input)
    display(localStatsWidget.submit_button)
    # Runs all statistics at once
    localStatsWidget.submit_button.on_click(display_all_stats)
    for out in localStatsWidget.outputs:
        display(out)

In [ ]:
class MenuWidgets:
    def __init__(self):
        self.output_area = widgets.Output(
            layout={"border": "3px solid black"}
        )
        self.specs = ["Local Statistics", "Global Statistics", "Transactions in Warehouse", "Part Prices", "Parts Supplied by Suppliers", "Parts in Warehouse"]

        self.spec_selection = widgets.Dropdown(
            options=self.specs,
            value = self.specs[0],
            description="Select view:"
        )

        self.spec_selection.observe(self.on_selection_change)

    def getSpec(self,selected_spec):
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            display_functions = {
                self.specs[0]: displayLocalStatsWidget,
                self.specs[1]: displayGlobalStatisticsWidget,
                self.specs[2]: displayTransactionWidget,
                self.specs[3]: displayPartPriceWidget,
                self.specs[4]: displaySuppliedParts,
                self.specs[5]: displayWarehouseWidgets,
            }
            
            display_markdowns = {
                self.specs[0]: (Markdown("## Local Statistics")),
                self.specs[1]: (Markdown("## Global Statistics")),
                self.specs[2]: (Markdown("## Transactions in each Warehouse")),
                self.specs[3]: (Markdown("## All Part Prices")),
                self.specs[4]: (Markdown("## Parts Supplied")),
                self.specs[5]: (Markdown("## Parts in a Warehouse")),
            }

            display_function = display_functions.get(selected_spec, None)
            display_markdown = display_markdowns.get(selected_spec, None)
            if display_function is not None:
                display(display_markdown)
                display_function()
            else:
                print(selected_spec)

    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSpec(selected_supplier)

In [ ]:
menuWidget = MenuWidgets()
menuWidget.getSpec(menuWidget.spec_selection.value)
menuWidget.spec_selection.observe(menuWidget.on_selection_change)
display(menuWidget.spec_selection)
display(menuWidget.output_area)